#### CSE4020 Final Project
Grant Butler | [gbutler2020@my.fit.edu](mailto:gbutler2020@my.fit.edu)

##### Task 01: Creating and Importing Tables with `dbdiagram`

Diagram in `dbdiagram`:

![](assets/db_diagram.png)

In [8]:
from IPython.display import display, Markdown
with open("er_diagram.sql") as f:
    for line in f:
        display(Markdown("```sql\n" + line.rstrip() + "\n```\n"))

```sql
CREATE DATABASE IF NOT EXISTS imdb_movies;
```


```sql
USE imdb_movies;
```


```sql
CREATE TABLE `movies_directors` (
```


```sql
  `director_id` int(11),
```


```sql
  `movie_id` int(11),
```


...

```sql
ALTER TABLE `directors_genres` ADD FOREIGN KEY (`director_id`) REFERENCES `directors` (`id`);
```


```sql
ALTER TABLE `roles` ADD FOREIGN KEY (`actor_id`) REFERENCES `actors` (`id`);
```


```sql
ALTER TABLE `roles` ADD FOREIGN KEY (`movie_id`) REFERENCES `movies` (`id`);
```


```sql
ALTER TABLE `movies_genres` ADD FOREIGN KEY (`movie_id`) REFERENCES `movies` (`id`);
```


```sql
ALTER TABLE `movies_directors` ADD FOREIGN KEY (`director_id`) REFERENCES `directors` (`id`);
```


##### Task 02: Connect and Add Tables

Connecting to RDS:

In [9]:
# - - - - - - - - - #
# connecting to RDS #
# - - - - - - - - - #

# imports
import os
import json
import mysql.connector as connector

# loading credentials from json
f = open("endpoint.json")
creds = json.load(f)

# making dict of credentials to log into the RDS server
config = {
    'user': creds["username"],
    'password': creds["password"],
    'host': creds["endpoint"],
    'database': creds["db_name"]
}

# connecting to endpoint with unpacking ( ⌐■-■)
conn = connector.connect(**config, autocommit=True)

if conn.is_connected():
    cursor = conn.cursor()

Adding Database from DDL File:

In [10]:
cursor.execute(open("er_diagram.sql").read(), multi=True)
conn.commit()

##### Task 03: Add Data Into DB

In [18]:
# - - - - - - - - - - - - - - - #
# inserting ddls into database  #
# - - - - - - - - - - - - - - - #

try: 
    if conn.is_connected():
        cursor2 = conn.cursor()

    for file in os.listdir('data/'):
        
        # method 0
        ddl = open('data/' + file)
        queries = ddl.read()
        cursor.execute(queries, multi=True)

        # method 1
        # with open('data/' + file) as f:
        #     for line in f:
        #         cursor2.execute(line.rstrip())

        # method 2
        # for result in cursor2.execute(open("data/" + file).read(), multi=True):
        #     print(result)

        print(f"{cursor2.rowcount} rows added to table {os.path.splitext(file)[0]}")

        conn.commit()

except connector.Error as error:
    conn.rollback()

-1 rows added to table actors
-1 rows added to table directors
-1 rows added to table directors_genres
-1 rows added to table movies
-1 rows added to table roles
-1 rows added to table movies_directors
-1 rows added to table movies_genres


##### Task 04: Execute Query for Not Null Ranked Movies

In [13]:
query = "SELECT COUNT(*) FROM movies WHERE rank IS NOT NULL;"

for result in cursor.execute(query, multi=True):
    print(result)

ProgrammingError: 1064 (42000): You have an error in your SQL syntax; check the manual that corresponds to your MySQL server version for the right syntax to use near 'IS NOT NULL' at line 1